## Implémentation

In [1]:
import sys

sys.path.append('../..')

In [2]:
from maths.montecarlo.processes.brownianmotions import GeometricBrownianMotion

In [3]:
step = 1 / 360.
maturity = 5.

In [4]:
x = [100]
d = [0.5]
v = [0.2]

time_grid = GeometricBrownianMotion.generate_time_grid(0, maturity, step)

udl = GeometricBrownianMotion(time_grid, x, d, v)

In [5]:
from finance.discountfactor import ConstantRateDiscountFactor 

r = 0.03
discount = ConstantRateDiscountFactor(r)

In [6]:
from finance.products.european.swap import (
    SwapContract,
)

delta = 0.25
coupon_dates = SwapContract.generate_payment_dates(0, maturity, delta)

swap = SwapContract(udl, discount, coupon_dates)

In [7]:
from risk.basel.eee import BlackScholesSwapVaREffExpectExposure

eee_mgr = BlackScholesSwapVaREffExpectExposure(swap)

In [8]:
import numpy

cm_positions = numpy.reshape([1., -1.], (2, 1))
cm_positions

array([[ 1.],
       [-1.]])

In [9]:
kwargs = {'t':0., 'risk_period': 6./360., 'alpha': 0.95}
eee_mgr(1., cm_positions, **kwargs)

array([[ 19.59586403],
       [  0.93720859]])

### OK voir l'autre NB

## Comparaison des signes avec l'implémentation courante

In [10]:
from risk.exposures import EuropeanVaRGeomBrownianExposure

quantile_df = 0.80

exposure = EuropeanVaRGeomBrownianExposure(swap, discount, d, v)

In [11]:
from finance.portfolio import CCPPortfolio

port = CCPPortfolio(cm_positions, [1.], [swap], [exposure])

In [12]:
print port.positions[0], port.compute_exposure(0., conf_level=0.95, risk_period=5./360., towards_=0)
print port.positions[1], port.compute_exposure(0., conf_level=0.95, risk_period=5./360., towards_=1)

[-1.] [ 14.39786516]
[ 1.] [ 19.79509752]


In [13]:
from risk.basel.exposures import BaselExposureAtDefault

ead_mgr = BaselExposureAtDefault(port, [eee_mgr])

In [14]:
ead_mgr(t=0., epsilon=1./12, risk_period=6./360., alpha=0.95)

# alpha=0.95:
# array([[  1.47966525],
#        [ 30.93795712]])

array([[  1.47966525],
       [ 30.93795712]])

In [15]:
ead_mgr(t=0., epsilon=1./12, risk_period=6./360., alpha=0.96)

# alpha=0.96
# array([[  1.14059146],
#       [ 30.27820753]])

array([[  1.14059146],
       [ 30.27820753]])

In [16]:
kwargs = {'t':0., 'risk_period': 6./360., 'alpha': 0.95}
eee_mgr(1., cm_positions, **kwargs)

array([[ 19.59586403],
       [  0.93720859]])

In [17]:
kwargs = {'t':0., 'risk_period': 6./360., 'alpha': 0.96}
eee_mgr(1., cm_positions, **kwargs)

array([[ 19.17798372],
       [  0.72244185]])